In [1]:
import xarray as xr
import numpy as np
import matplotlib.pylab as plt
import matplotlib.cm as cm
import glob
import intake
import dask
from scipy.interpolate import NearestNDInterpolator
from scipy.spatial import cKDTree

/sw/spack-levante/mambaforge-23.1.0-1-Linux-x86_64-3boc6i/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
import xarray as xr
import numpy as np
import intake

eerie_cat=intake.open_catalog("https://raw.githubusercontent.com/eerie-project/intake_catalogues/main/eerie.yaml")

model = 'ifs-fesom2-sr'
expid = 'hist-1950'  # 'highres-future-ssp245', 'hist-1950', 'eerie-control-1950', 'eerie-spinup-1950'
version = 'v20240304'    #
realm='atmos'  # 'ocean' , 'land', 'atmos'
gridspec = 'native' # 'native', 'gr025'

cat_data=eerie_cat['dkrz.disk.model-output'][model][expid][version][realm][gridspec]
print(list(cat_data))

# ds = cat_data['2D_daily_avg'].to_dask()
# ds

['2D_daily_avg', '2D_daily_max', '2D_daily_min', '2D_monthly_avg', '2D_monthly_max', '2D_monthly_min', '3D_daily_avg', '3D_monthly_avg', '3D_6hourly', '2D_6hourly_accumulated', '2D_6hourly_instant']


In [3]:
ds = cat_data['2D_daily_avg'].to_dask()
ds

<xarray.Dataset>
Dimensions:    (time: 23741, value: 6599680)
Coordinates:
    lat        (value) float64 dask.array<chunksize=(6599680,), meta=np.ndarray>
    lon        (value) float64 dask.array<chunksize=(6599680,), meta=np.ndarray>
  * time       (time) datetime64[ns] 1950-01-01T12:00:00 ... 2014-12-31T12:00:00
Dimensions without coordinates: value
Data variables: (12/43)
    cprate     (time, value) float64 dask.array<chunksize=(2, 6599680), meta=np.ndarray>
    erate      (time, value) float64 dask.array<chunksize=(2, 6599680), meta=np.ndarray>
    m10u       (time, value) float64 dask.array<chunksize=(2, 6599680), meta=np.ndarray>
    m10v       (time, value) float64 dask.array<chunksize=(2, 6599680), meta=np.ndarray>
    m2d        (time, value) float64 dask.array<chunksize=(2, 6599680), meta=np.ndarray>
    m2sh       (time, value) float64 dask.array<chunksize=(2, 6599680), meta=np.ndarray>
    ...         ...
    mtnlwrf    (time, value) float64 dask.array<chunksize=(2, 6599680), meta=np.ndarray>
    mtnlwrfcs  (time, value) float64 dask.array<chunksize=(2, 6599680), meta=np.ndarray>
    mtnswrf    (time, value) float64 dask.array<chunksize=(2, 6599680), meta=np.ndarray>
    mtnswrfcs  (time, value) float64 dask.array<chunksize=(2, 6599680), meta=np.ndarray>
    mtsfr      (time, value) float64 dask.array<chunksize=(2, 6599680), meta=np.ndarray>
    tprate     (time, value) float64 dask.array<chunksize=(2, 6599680), meta=np.ndarray>
Attributes: (12/36)
    edition:               2
    centre:                ecmf
    centreDescription:     European Centre for Medium-Range Weather Forecasts
    subCentre:             0
    history:               🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan...
    Conventions:           CF-1.7 CMIP-6.2
    ...                    ...
    parent_activity_id:    HighResMIP
    sub_experiment_id:     none
    experiment:            coupled historical 1950-2014
    source:                IFS-FESOM2-SR (2023): \naerosol: none, prescribed ...
    institution:           Alfred Wegener Institute, Helmholtz Centre for Pol...
    sub_experiment:        none

In [4]:
from dask.distributed import Client
import dask
dask.config.config.get('distributed').get('dashboard').update({'link':'{JUPYTERHUB_SERVICE_PREFIX}/proxy/{port}/status'})
dask.config.set({'temporary_directory': '/scratch/a/a270088/dask/'})
# client = Client(n_workers=5, threads_per_worker=1, memory_limit='50GB')
client = Client(n_workers=5, threads_per_worker=1, memory_limit='20GB')
client

/sw/spack-levante/mambaforge-23.1.0-1-Linux-x86_64-3boc6i/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/sw/spack-levante/mambaforge-23.1.0-1-Linux-x86_64-3boc6i/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/sw/spack-levante/mambaforge-23.1.0-1-Linux-x86_64-3boc6i/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/sw/spack-levante/mambaforge-23.1.0-1-Linux-x86_64-3boc6i/lib/python3.10/site-packages/pandas/core/computa

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/a270088/levante-spawner-preset//proxy/8787/status,
Dashboard: /user/a270088/levante-spawner-preset//proxy/8787/status,Workers: 5
Total threads: 5,Total memory: 93.13 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46297,Workers: 5
Dashboard: /user/a270088/levante-spawner-preset//proxy/8787/status,Total threads: 5
Started: Just now,Total memory: 93.13 GiB
Comm: tcp://127.0.0.1:35617,Total threads: 1
Dashboard: /user/a270088/levante-spawner-preset//proxy/36353/status,Memory: 18.63 GiB
Nanny: tcp://127.0.0.1:37195,


In [5]:
%%time
ds['mean2t'].sel(time=slice('1950','1951')).to_netcdf('/work/ab0995/a270088/ICCPhackathon/test.nc')

CPU times: user 8.89 s, sys: 3.33 s, total: 12.2 s
Wall time: 1min 25s


In [6]:
%%time
temp_std = ds['mean2t'].sel(time=slice('1950','1951')).std(dim='time').compute()
